# Testing out Local LLMs

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

ModuleNotFoundError: No module named 'langchain_ollama'

In [7]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [3]:
# Loading the model
# llm = ChatOllama(model="llama3.2:1b", temperature=0)
llm = ChatOllama(model="llama3.2", temperature=0)

In [6]:
initialSystemMessage = '''You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.
'''


messages = [
    ("system",initialSystemMessage),
    ("human", "Pay my bill")
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="To proceed with paying your bill, I need to know a few details first.\n\nCan you please tell me the number of the bill you'd like to pay?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-12T22:14:36.447587Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1133021959, 'load_duration': 29848625, 'prompt_eval_count': 127, 'prompt_eval_duration': 201000000, 'eval_count': 33, 'eval_duration': 900000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-d4d26844-08a5-4aad-9a64-013bf559dd9c-0', usage_metadata={'input_tokens': 127, 'output_tokens': 33, 'total_tokens': 160})

## Testing Tools

### Calculator Tools

In [7]:
@tool
def add(a: int, b: int) -> int:
    """Add (+) two numbers"""
    return a+b
    
@tool
def sub(a: int, b: int) -> int:
    """Subtract (-) two numbers"""
    return a-b
    
@tool
def mul(a: int, b: int) -> int:
    """Multiply (*, x) two numbers"""
    return a*b
    
@tool
def div(a: int, b: int) -> int:
    """Divide (/) two numbers"""
    return a/b

@tool
def fetchUserDets(name: str) -> str:
    """Function to fetch user details. Input is name of the user. Output relevant details."""
    # hmap = 
    pass



tools = [mul, div, add, sub]
toolsMap = {'add':add, "sub":sub, "mul":mul, "div":div}

llmTools = llm.bind_tools(tools)


In [8]:
query = "Add 3 and 4?"
query = "What is 3 multiplied 12? Also, what is 11 + 49?"

# llmTools.invoke(query).tool_calls

messages = [
    ("system","You are a helpful assistant. be kind"),
    ("human", "Tell me the answer to 3 * 12? Also, what is 11 - 49?")
]
ai_msg = llmTools.invoke(messages)
messages.append(ai_msg)
ai_msg

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-09T22:54:25.2212492Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2882431800, 'load_duration': 53992000, 'prompt_eval_count': 353, 'prompt_eval_duration': 23000000, 'eval_count': 44, 'eval_duration': 2799000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c5d7bdb8-835e-4270-ae67-f75cf434ef9a-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 12}, 'id': 'f1203f1f-0351-4a91-830b-16f2f5f301be', 'type': 'tool_call'}, {'name': 'sub', 'args': {'a': 11, 'b': 49}, 'id': '7fb06331-ac6d-4474-8cb6-b2c66d70bfb7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 353, 'output_tokens': 44, 'total_tokens': 397})

In [9]:
ai_msg.tool_calls

[{'name': 'mul',
  'args': {'a': 3, 'b': 12},
  'id': 'f1203f1f-0351-4a91-830b-16f2f5f301be',
  'type': 'tool_call'},
 {'name': 'sub',
  'args': {'a': 11, 'b': 49},
  'id': '7fb06331-ac6d-4474-8cb6-b2c66d70bfb7',
  'type': 'tool_call'}]

In [10]:
messages

[('system', 'You are a helpful assistant. be kind'),
 ('human', 'Tell me the answer to 3 * 12? Also, what is 11 - 49?'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-09T22:54:25.2212492Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2882431800, 'load_duration': 53992000, 'prompt_eval_count': 353, 'prompt_eval_duration': 23000000, 'eval_count': 44, 'eval_duration': 2799000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c5d7bdb8-835e-4270-ae67-f75cf434ef9a-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 12}, 'id': 'f1203f1f-0351-4a91-830b-16f2f5f301be', 'type': 'tool_call'}, {'name': 'sub', 'args': {'a': 11, 'b': 49}, 'id': '7fb06331-ac6d-4474-8cb6-b2c66d70bfb7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 353, 'output_tokens': 44, 'total_tokens': 397})]

In [11]:
for toolCall in ai_msg.tool_calls:
    toolSelected = toolsMap[toolCall['name']]
    toolMsg = toolSelected.invoke(toolCall)
    messages.append(toolMsg)
    
messages

[('system', 'You are a helpful assistant. be kind'),
 ('human', 'Tell me the answer to 3 * 12? Also, what is 11 - 49?'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-09T22:54:25.2212492Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2882431800, 'load_duration': 53992000, 'prompt_eval_count': 353, 'prompt_eval_duration': 23000000, 'eval_count': 44, 'eval_duration': 2799000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c5d7bdb8-835e-4270-ae67-f75cf434ef9a-0', tool_calls=[{'name': 'mul', 'args': {'a': 3, 'b': 12}, 'id': 'f1203f1f-0351-4a91-830b-16f2f5f301be', 'type': 'tool_call'}, {'name': 'sub', 'args': {'a': 11, 'b': 49}, 'id': '7fb06331-ac6d-4474-8cb6-b2c66d70bfb7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 353, 'output_tokens': 44, 'total_tokens': 397}),
 ToolMessage(content='36', name='mul', tool_call_id='f1203f1f-0351-4a91-830b-16f2f5f301be'),
 ToolMe

In [12]:
ai_msg = llmTools.invoke(messages)

In [13]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


In [14]:
ai_msg

AIMessage(content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-09T22:54:26.2939606Z', 'done': True, 'done_reason': 'stop', 'total_duration': 981457900, 'load_duration': 57474600, 'prompt_eval_count': 137, 'prompt_eval_duration': 10000000, 'eval_count': 25, 'eval_duration': 907000000, 'message': Message(role='assistant', content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', images=None, tool_calls=None)}, id='run-194d59fa-b28f-4ec4-9bf4-00d648665da4-0', usage_metadata={'input_tokens': 137, 'output_tokens': 25, 'total_tokens': 162})

In [15]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


---

In [1]:
!which python

/Users/asutoshdalei/Desktop/Work/solar/bin/python


In [2]:
import sys
sys.executable

'/Users/asutoshdalei/Desktop/Work/solar/bin/python3.11'

In [5]:
initialSystemMessage = '''You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.
'''


In [6]:
print(initialSystemMessage)

You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.



----

In [10]:
import json

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Importing model
llm = ChatOllama(model="llama3.2", temperature=0)

In [11]:
billDB = {
    9182:{'Customer Name':'Asutosh Dalei','service provider':'TS Elec Board','unit':32,'Amount':341, "Due Date":'10/01/2025', 'status':'Paid'},
    1928:{'Customer Name':'Asutosh Dalei','service provider':'TS Elec Board','unit':37,'Amount':547, "Due Date":'11/02/2025','status':'Unpaid'},
    1038:{'Customer Name':'Asutosh Dalei','service provider':'TS Elec Board','unit':23,'Amount':298, "Due Date":'09/10/2025','status':'Paid'}
}


initialSystemMessage = '''You are an excellent virtual assistant. Your name is PayLLM. In the first user interaction, respond directly without calling any tools.
You should strictly follow the following steps:
0. Ask the user if they want to pay any bill.
1. Ask the user the bill number.
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never use external knowledge, assumptions or information beyond what is explicitly shared or recieved.
Strictly do not call tools unless absolutely needed.

Example:
User: Hello
PayLLM: Hi. To help with your payments, please provide your bill number.
User: 12345.
PayLLM: Sure, should I fetch the bill?
User: Yes, sure.
PayLLM: The bill amount is 145
User: Okay. Go ahead and pay it.
PayLLM: Sure.
PayLLM: The bill is paid.
'''

In [6]:
@tool
def payBill(billNumber: str) -> str:
    """Function to pay bill using bill number.
    Args:
        billNumber: Bill Number (String). Output: Bill paid status
    """
    print(billNumber, type(billNumber))
    if billDB[billNumber]['status'] == 'Paid':
        return "Bill paid already"
    billDB[billNumber]['status'] = 'Paid'
    return "Bill paid successfully"

serviceTools = [payBill]
serviceToolsMap = {"payBill": payBill}

firstInteraction = True

In [ ]:
def event(llm):
    serviceMessages = [SystemMessage(content = initialSystemMessage), HumanMessage(content='Help me pay my bill. Ask me my bill number.')]
    llmService = llm
    aiMsgSer = llmService.invoke(serviceMessages)
    while True:
        if firstInteraction:
                print(f"AI Response:\n --> {aiMsgSer.content}")
                firstInteraction = False
                llmService = llm.bind_tools(serviceTools)

        if aiMsgSer.tool_calls:
            for toolCallSer in aiMsgSer.tool_calls:

                toolSelected = serviceToolsMap[toolCallSer['name']]
                toolMsg = toolSelected.invoke(toolCallSer)
                print(toolMsg)
                serviceMessages.append(toolMsg)


            aiMsgSer = llmService.invoke(serviceMessages)
            serviceMessages.append(aiMsgSer)
            print(f"AI Response:\n --> {aiMsgSer.content}")
            continue
        userInput = input("User Input:\n -->")
        if userInput == "/end":
            break
        serviceMessages.append(HumanMessage(content = userInput))
        aiMsgSer = llmService.invoke(serviceMessages)
        if aiMsgSer.content != '':
            print(f"AI Response:\n--> {aiMsgSer.content}")
        serviceMessages.append(HumanMessage(content = userInput))

In [ ]:

try:
    event()
except Exception as e:
    print(e)

cannot access local variable 'llmService' where it is not associated with a value
